# This notebook illsutrates how to evaluate the performance of runs

The main function used is evaluate_project from evaluate.py. To use the function 2 main thigs need to be defined:
- which dataset directories to use:
    - `data_dir` = for test set to use 
- which models to evaluate. To supply these **ONE** of 3 options need to be done:
    - `run_dir_list`  = list of pre-defined runs 
    - `project_dir` = a directory that contains subdirectories of models that will be evaluated
    - `wandb_project_name` = project completed in WandB. If using this option must also provide `wandb_dir` = a directory path where the outputs of the project are saved

Optionally output path `output_dir`, filename `output_prefix` can be defined and `batch_size` can be defined. Set batch size to smaller values for bigger models (at the cost of longer duration)

The outputs are:
- csv of the evaluation results wih 2 rows per target of a model corresponding to scaled or raw results (so the csv is made up of *N(models) * N(targets) * 2 rows*


In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../scripts')
import evaluate
import pandas as pd

In [2]:
%%time
# use list of run directories
output_dir = 'tutorial_outputs/'
evaluate.evaluate_project(data_dir='../data/tfr_datasets/i_2048_w_1/',
                          run_dir_list=['tutorial_outputs'], output_dir=output_dir,
                         batch_size=512)

USING PREDEFINED LIST OF RUNS
tutorial_outputs
CPU times: user 9.18 s, sys: 14.1 s, total: 23.3 s
Wall time: 13.3 s


In [3]:
evaluation_results = pd.read_csv(output_dir+'/evaluation_results.csv', index_col='index') # get results from default output path
evaluation_results.head(2)

,mse,js_per_seq,js_conc,poiss,pr_corr,sp_corr,targets,pred type,eval type,alpha,...,metrics,model_fn,num_epochs,record_test,rev_comp,shuffle,sigma,verbose,run_dir,scaling_factors
index,,,,,,,,,,,,,,,,,,,,,
0,0.642281,0.505163,0.615927,0.985061,0.310499,0.193765,PC-3_replicate_1,raw,whole,False,...,"['mse', 'pearsonr', 'poisson']",basenjimod,2,False,True,True,20,True,tutorial_outputs,NaN
1,0.396133,0.480455,0.497146,0.983344,0.429005,0.341531,Panc1_replicate_1,raw,whole,False,...,"['mse', 'pearsonr', 'poisson']",basenjimod,2,False,True,True,20,True,tutorial_outputs,NaN


In [4]:
eval_type = 'whole'
pred_type = 'raw'

raw_whole_results = evaluation_results[(evaluation_results['pred type']==pred_type)&
                                      (evaluation_results['eval type']==eval_type)]
print('Average Pearson\'s r per run')
raw_whole_results.groupby('run_dir').mean()

Average Pearson's r per run


,mse,js_per_seq,js_conc,poiss,pr_corr,sp_corr,alpha,batch_size,bin_size,crop,...,log_wandb,lr_decay,lr_patience,num_epochs,record_test,rev_comp,shuffle,sigma,verbose,scaling_factors
run_dir,,,,,,,,,,,,,,,,,,,,,
tutorial_outputs,0.519207,0.492809,0.556537,0.984202,0.369752,0.267648,False,64,32,True,...,False,0.3,10,2,False,True,True,20,True,NaN


In [5]:
# per cell line pearson r results
raw_whole_results.sort_values('pr_corr')[['targets', 'pr_corr']]

,targets,pr_corr
index,,
0,PC-3_replicate_1,0.310499
1,Panc1_replicate_1,0.429005
